# Train prediction models for texture 
* Convolutional neural network over samples; input frequency spectra of microphone data                
* training data concatenated over speed leaving one speed out but retained for testing
* therefore total of n_speed models (here 10), one per speed
* i.e. prediction of texture independent of held-out speed
* data augemnted by stretching or squashing the ffts

To run, first edit dir_data to path where data is stored; spaital/process_vibration in this directory should be run first.

Because of the large amount of data, a temporary folder is used for the training data, so it can be deleted after training the dynamic model. 

In [1]:
import os, pickle
import numpy as np
from scipy import signal

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [2]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-textures-jrsi2022"
dir_temp = os.environ["TEMPPATH"] + r"/vibration"

n_textures = 13
n_speeds = 10
fs = 44100 # sampling rate per second
n_data = 119 # number of data segments per speed & texture
t_segment = 1 # duration of data segment
n_samples = t_segment*fs # number of samples per data segment
n_freqs = 4000
n_features = 200 # freqs to keep as features 
n_train = n_data*(n_speeds-1)*n_textures

In [3]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session
from keras import optimizers, regularizers, callbacks
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, BatchNormalization
from numpy.random import seed
from tensorflow import set_random_seed

C:\Users\nl13426\anaconda3\envs\tactip\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nl13426\anaconda3\envs\tactip\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nl13426\anaconda3\envs\tactip\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nl13426\anaconda3\envs\tactip\lib\site-packages\tensorf

In [4]:
def define_model(X_train, y_train, X_val, y_val, es, cp):  
    clear_session()
    
    config = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)) # , device_count = {'GPU': 1})
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    set_session(session)
    
    seed(1)
    set_random_seed(2)
    
    model = Sequential()
    model.add(Conv1D(128, 5, activation='relu', padding = "same", input_shape=(n_features,1)))
    model.add(MaxPooling1D(3))
    model.add(BatchNormalization())
    model.add(Conv1D(128, 5, activation='relu', padding = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 5, activation='relu', padding = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 5, activation='relu', padding = "valid"))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(32, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(Dense(n_textures, activation='softmax'))#)kernel_regularizer=regularizers.l2(0.005), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-4), metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=150, batch_size=64, shuffle=True, callbacks=[es,cp])

In [6]:
for i in range(n_speeds):
    data_set = dir_temp + rf"/{i}"
    X_train = open_obj(data_set + r"/X_train") 
    y_train = open_obj(data_set + r"/y_train")  
    X_val = open_obj(data_set + r"/X_val")
    y_val = open_obj(data_set + r"/y_val")

    X_train = np.expand_dims(X_train[:,:n_features], axis=2)
    X_val = np.expand_dims(X_val[:,:n_features], axis=2)

    dir_model = dir_data + rf"/models/vibration/{i}"
    os.makedirs(dir_model)
    cp = callbacks.ModelCheckpoint(dir_model + r"/model_{epoch:02d}_{val_accuracy:.2f}.hdf5", monitor='val_accuracy', save_best_only=True)
    es = callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    define_model(X_train, y_train, X_val, y_val, es, cp)



Train on 13923 samples, validate on 767 samples
Epoch 1/150
13923/13923 [==============================] - 8s 566us/step - loss: 2.0053 - accuracy: 0.4398 - val_loss: 3.9021 - val_accuracy: 0.0769
Epoch 2/150
13923/13923 [==============================] - 3s 201us/step - loss: 1.1515 - accuracy: 0.7361 - val_loss: 6.0583 - val_accuracy: 0.0769
Epoch 3/150
13923/13923 [==============================] - 3s 199us/step - loss: 0.8158 - accuracy: 0.8461 - val_loss: 6.6924 - val_accuracy: 0.0769
Epoch 4/150
13923/13923 [==============================] - 3s 199us/step - loss: 0.6284 - accuracy: 0.9015 - val_loss: 3.7085 - val_accuracy: 0.1186
Epoch 5/150
13923/13923 [==============================] - 3s 199us/step - loss: 0.5228 - accuracy: 0.9316 - val_loss: 3.6717 - val_accuracy: 0.1160
Epoch 6/150
13923/13923 [==============================] - 3s 198us/step - loss: 0.4471 - accuracy: 0.9510 - val_loss: 3.8655 - val_accuracy: 0.1199
Epoch 7/150
13923/13923 [==============================]

In [7]:
n_aug = 3 # half stretching/squashing - simulate faster/slower speed

def augment(X_train, y_train):
    X_train_aug = np.zeros((n_train*2*n_aug, n_features))
    y_train_aug = np.zeros((n_train*2*n_aug, n_textures))
    stretch = np.concatenate([np.random.uniform(.5,1,(n_aug,n_train)), np.random.uniform(1,2,(n_aug,n_train))])
    for i in range(n_train):
        for j in range(2*n_aug):
            X_train_aug[i*2*n_aug+j,:] = signal.resample(X_train[i,:], int(stretch[j,i]*n_freqs))[:n_features] 
            y_train_aug[i*2*n_aug+j,:] = y_train[i,:]
    return X_train_aug, y_train_aug
    

In [8]:
for i in range(n_speeds):
    data_set = dir_temp + rf"/{i}"
    X_train = open_obj(data_set + r"/X_train") 
    y_train = open_obj(data_set + r"/y_train")  
    X_val = open_obj(data_set + r"/X_val")
    y_val = open_obj(data_set + r"/y_val")

    X_train, y_train = augment(X_train, y_train)   # augmented

    X_train = np.expand_dims(X_train[:,:n_features], axis=2)
    X_val = np.expand_dims(X_val[:,:n_features], axis=2)

    dir_model = dir_data + rf"/models/vibration_augmented/{i}"
    os.makedirs(dir_model)
    cp = callbacks.ModelCheckpoint(dir_model + r"/model_{epoch:02d}_{val_accuracy:.2f}.hdf5", monitor='val_accuracy', save_best_only=True)
    es = callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    define_model(X_train, y_train, X_val, y_val, es, cp)

Train on 83538 samples, validate on 767 samples
Epoch 1/150
83538/83538 [==============================] - 16s 192us/step - loss: 1.4722 - accuracy: 0.5858 - val_loss: 2.3852 - val_accuracy: 0.2386
Epoch 2/150
83538/83538 [==============================] - 15s 178us/step - loss: 0.7681 - accuracy: 0.8153 - val_loss: 2.2558 - val_accuracy: 0.2751
Epoch 3/150
83538/83538 [==============================] - 15s 177us/step - loss: 0.5639 - accuracy: 0.8747 - val_loss: 2.0744 - val_accuracy: 0.3233
Epoch 4/150
83538/83538 [==============================] - 15s 176us/step - loss: 0.4457 - accuracy: 0.9047 - val_loss: 2.4899 - val_accuracy: 0.2568
Epoch 5/150
83538/83538 [==============================] - 15s 178us/step - loss: 0.3680 - accuracy: 0.9216 - val_loss: 2.1872 - val_accuracy: 0.2973
Epoch 6/150
83538/83538 [==============================] - 15s 175us/step - loss: 0.3093 - accuracy: 0.9356 - val_loss: 2.0405 - val_accuracy: 0.3794
Epoch 7/150
83538/83538 [===========================